In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

_BATCH_NORM_DECAY = 0.997
_BATCH_NORM_EPSILON = 1e-5

def batch_norm(inputs, training):
  return tf.layers.batch_normalization(
      inputs=inputs, axis=3,
      momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON, center=True,
      scale=True, training=training, fused=True)

def fixed_padding(inputs, kernel_size):
  """Pads the input along the spatial dimensions independently of input size."""
  pad_total = kernel_size - 1
  pad_beg = pad_total // 2
  pad_end = pad_total - pad_beg

  padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end],
                                    [pad_beg, pad_end], [0, 0]])
  return padded_inputs


def conv2d_fixed_padding(inputs, filters, kernel_size, strides):
  """Strided 2-D convolution with explicit padding."""
  if strides > 1:
    inputs = fixed_padding(inputs, kernel_size)

  return tf.layers.conv2d(
      inputs=inputs, filters=filters, kernel_size=kernel_size, strides=strides,
      padding=('SAME' if strides == 1 else 'VALID'), use_bias=False,
      kernel_initializer=tf.variance_scaling_initializer())

def resnetUnit(input, filters, kernel_size, strides, bottleneck, training):
  """A resnet50 unit with two branches."""
  shortcut = input
  if bottleneck:
    shortcut = conv2d_fixed_padding(
      inputs=shortcut, filters=filters[3], strides=strides[3], kernel_size=kernel_size[3])
    shortcut = batch_norm(shortcut, training)
  
  input = conv2d_fixed_padding(
      inputs=input, filters=filters[0], strides=strides[0], kernel_size=kernel_size[0])
  input = batch_norm(input, training)
  input = conv2d_fixed_padding(
      inputs=input, filters=filters[1], strides=strides[1], kernel_size=kernel_size[1])
  input = batch_norm(input, training)
  input = conv2d_fixed_padding(
      inputs=input, filters=filters[2], strides=strides[2], kernel_size=kernel_size[2])
  input = batch_norm(input, training)
  
  return tf.nn.relu(input + shortcut)
  

def resnet50(features, mode):
  training = (mode == "TRAIN")

  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  #input_layer = tf.reshape(features["x"], [-1, 224, 224, 3])
  input_layer = tf.reshape(features, [-1, 28, 28, 1])


  # Convolutional Layer #1
  conv1 = conv2d_fixed_padding(
      inputs=input_layer, filters=64, strides=2, kernel_size=7)
  conv1_norm = batch_norm(conv1, training)
  conv1_relu = tf.nn.relu(conv1_norm)
  pool1 = tf.layers.max_pooling2d(
    inputs=conv1_relu, pool_size=3, strides=2, padding='SAME',
    data_format='channels_last')
  
  kernel_size = [1, 3, 1, 1]
  filters_2 = [64, 64, 256, 256]
  strides_2 = [1, 1, 1, 1]
  
  #res2a
  res2a_relu = resnetUnit(pool1, filters_2, kernel_size, strides_2,
                          bottleneck = True, training = training)
  res2b_relu = resnetUnit(res2a_relu, filters_2, kernel_size, strides_2,
                          bottleneck = False, training = training)
  res2c_relu = resnetUnit(res2b_relu, filters_2, kernel_size, strides_2,
                          bottleneck = False, training = training)
  
  #res3a
  filters_3 = [128, 128, 512, 512]
  strides_3 = [2, 1, 1, 2]
  res3a_relu = resnetUnit(res2c_relu, filters_3, kernel_size, strides_3,
                          bottleneck = True, training = training)
  res3b_relu = resnetUnit(res3a_relu, filters_3, kernel_size, strides_2,
                          bottleneck = False, training = training)
  res3c_relu = resnetUnit(res3b_relu, filters_3, kernel_size, strides_2,
                          bottleneck = False, training = training)
  res3d_relu = resnetUnit(res3c_relu, filters_3, kernel_size, strides_2,
                          bottleneck = False, training = training)
  
  #res4a
  filters_4 = [256, 256, 1024, 1024]
  res4a_relu = resnetUnit(res3d_relu, filters_4, kernel_size, strides_3,
                          bottleneck = True, training = training)
  res4b_relu = resnetUnit(res4a_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training)  
  res4c_relu = resnetUnit(res4b_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training)  
  res4d_relu = resnetUnit(res4c_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training)  
  res4e_relu = resnetUnit(res4d_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training)  
  res4f_relu = resnetUnit(res4e_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training) 
  
  #res5a
  filters_5 = [512, 512, 2048, 2048]
  res5a_relu = resnetUnit(res4f_relu, filters_5, kernel_size, strides_3,
                          bottleneck = True, training = training)
  res5b_relu = resnetUnit(res5a_relu, filters_5, kernel_size, strides_2,
                          bottleneck = False, training = training) 
  res5c_relu = resnetUnit(res5b_relu, filters_5, kernel_size, strides_2,
                          bottleneck = False, training = training) 

  # ResNet does an Average Pooling layer over pool_size.
  # Do a reduce_mean because it performs better than AveragePooling2D.
  axes = [1, 2]
  inputs = tf.reduce_mean(res5c_relu, axes, keepdims=True)
  inputs = tf.reshape(inputs, [-1, 1*1*2048])
  
  dense = tf.layers.dense(inputs=inputs, units=1000, activation=tf.nn.relu)
  dropout = tf.layers.dropout(inputs=dense, rate=0.5, training=training)

  logits = tf.layers.dense(inputs=dropout, units=10)
  
  return tf.nn.softmax(logits)

def loss(labels, logits):
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  return loss



def _generate_image_and_label_batch(image, label, min_queue_examples, batch_size):
  """Construct a queued batch of images and labels.
  Args:
    image: 3-D Tensor of [height, width, 3] of type.float32.
    label: 1-D Tensor of type.int32
    min_queue_examples: int32, minimum number of samples to retain
      in the queue that provides of batches of examples.
    batch_size: Number of images per batch.
    shuffle: boolean indicating whether to use a shuffling queue.
  Returns:
    images: Images. 4D tensor of [batch_size, height, width, 3] size.
    labels: Labels. 1D tensor of [batch_size] size.
  """
  # Create a queue that shuffles the examples, and then
  # read 'batch_size' images + labels from the example queue.
  num_preprocess_threads = 16
  images, label_batch = tf.train.shuffle_batch(
      [image, label],
      batch_size=batch_size,
      num_threads=num_preprocess_threads,
      capacity=min_queue_examples + 3 * batch_size,
      min_after_dequeue=min_queue_examples)

  return images, tf.reshape(label_batch, [batch_size])


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  y = tf.placeholder(tf.int32, [None])
  x = tf.placeholder(tf.float32, [100, 784])
  y_ = resnet50(features = x, mode= "TRAIN")
  crossEntropy = loss(y, y_)
  train_step = tf.train.GradientDescentOptimizer(0.005).minimize(crossEntropy)

  sess = tf.InteractiveSession()
  tf.global_variables_initializer().run()
  # Train
  for i in range(10):
    #batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: train_data, y_: train_labels})

  # Test trained model
  correct_prediction = tf.equal(tf.argmax(y, 1), y_)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print(sess.run(
      accuracy, feed_dict={
          x: mnist.test.images,
          y_: mnist.test.labels
      }))
  
  
if __name__ == "__main__":
  tf.app.run()


In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

_BATCH_NORM_DECAY = 0.997
_BATCH_NORM_EPSILON = 1e-5

def batch_norm(inputs, training):
  return tf.layers.batch_normalization(
      inputs=inputs, axis=3,
      momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON, center=True,
      scale=True, training=training, fused=True)

def fixed_padding(inputs, kernel_size):
  """Pads the input along the spatial dimensions independently of input size."""
  pad_total = kernel_size - 1
  pad_beg = pad_total // 2
  pad_end = pad_total - pad_beg

  padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end],
                                    [pad_beg, pad_end], [0, 0]])
  return padded_inputs


def conv2d_fixed_padding(inputs, filters, kernel_size, strides):
  """Strided 2-D convolution with explicit padding."""
  if strides > 1:
    inputs = fixed_padding(inputs, kernel_size)

  return tf.layers.conv2d(
      inputs=inputs, filters=filters, kernel_size=kernel_size, strides=strides,
      padding=('SAME' if strides == 1 else 'VALID'), use_bias=False,
      kernel_initializer=tf.variance_scaling_initializer())

def resnetUnit(input, filters, kernel_size, strides, bottleneck, training):
  """A resnet50 unit with two branches."""
  shortcut = input
  if bottleneck:
    shortcut = conv2d_fixed_padding(
      inputs=shortcut, filters=filters[3], strides=strides[3], kernel_size=kernel_size[3])
    shortcut = batch_norm(shortcut, training)
  
  input = conv2d_fixed_padding(
      inputs=input, filters=filters[0], strides=strides[0], kernel_size=kernel_size[0])
  input = batch_norm(input, training)
  input = conv2d_fixed_padding(
      inputs=input, filters=filters[1], strides=strides[1], kernel_size=kernel_size[1])
  input = batch_norm(input, training)
  input = conv2d_fixed_padding(
      inputs=input, filters=filters[2], strides=strides[2], kernel_size=kernel_size[2])
  input = batch_norm(input, training)
  
  return tf.nn.relu(input + shortcut)
  

def resnet50(features, labels, mode):
  training = (mode == tf.estimator.ModeKeys.TRAIN)

  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  #input_layer = tf.reshape(features["x"], [-1, 224, 224, 3])
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])


  # Convolutional Layer #1
  conv1 = conv2d_fixed_padding(
      inputs=input_layer, filters=64, strides=2, kernel_size=7)
  conv1_norm = batch_norm(conv1, training)
  conv1_relu = tf.nn.relu(conv1_norm)
  pool1 = tf.layers.max_pooling2d(
    inputs=conv1_relu, pool_size=3, strides=2, padding='SAME',
    data_format='channels_last')
  
  kernel_size = [1, 3, 1, 1]
  filters_2 = [64, 64, 256, 256]
  strides_2 = [1, 1, 1, 1]
  
  #res2a
  res2a_relu = resnetUnit(pool1, filters_2, kernel_size, strides_2,
                          bottleneck = True, training = training)
  res2b_relu = resnetUnit(res2a_relu, filters_2, kernel_size, strides_2,
                          bottleneck = False, training = training)
  res2c_relu = resnetUnit(res2b_relu, filters_2, kernel_size, strides_2,
                          bottleneck = False, training = training)
  
  filters_3 = [128, 128, 512, 512]
  strides_3 = [2, 1, 1, 2]
  res3a_relu = resnetUnit(res2c_relu, filters_3, kernel_size, strides_3,
                          bottleneck = True, training = training)
  res3b_relu = resnetUnit(res3a_relu, filters_3, kernel_size, strides_2,
                          bottleneck = False, training = training)
  res3c_relu = resnetUnit(res3b_relu, filters_3, kernel_size, strides_2,
                          bottleneck = False, training = training)
  res3d_relu = resnetUnit(res3c_relu, filters_3, kernel_size, strides_2,
                          bottleneck = False, training = training)
  
  filters_4 = [256, 256, 1024, 1024]
  res4a_relu = resnetUnit(res3d_relu, filters_4, kernel_size, strides_3,
                          bottleneck = True, training = training)
  res4b_relu = resnetUnit(res4a_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training)  
  res4c_relu = resnetUnit(res4b_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training)  
  res4d_relu = resnetUnit(res4c_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training)  
  res4e_relu = resnetUnit(res4d_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training)  
  res4f_relu = resnetUnit(res4e_relu, filters_4, kernel_size, strides_2,
                          bottleneck = False, training = training) 
  
  filters_5 = [512, 512, 2048, 2048]
  res5a_relu = resnetUnit(res4f_relu, filters_5, kernel_size, strides_3,
                          bottleneck = True, training = training)
  res5b_relu = resnetUnit(res5a_relu, filters_5, kernel_size, strides_2,
                          bottleneck = False, training = training) 
  res5c_relu = resnetUnit(res5b_relu, filters_5, kernel_size, strides_2,
                          bottleneck = False, training = training) 

  # ResNet does an Average Pooling layer over pool_size.
  # Do a reduce_mean because it performs better than AveragePooling2D.
  axes = [1, 2]
  inputs = tf.reduce_mean(res5c_relu, axes, keepdims=True)
  inputs = tf.reshape(inputs, [-1, 1*1*2048])
  
  dense = tf.layers.dense(inputs=inputs, units=1000, activation=tf.nn.relu)
  dropout = tf.layers.dropout(inputs=dense, rate=0.5, training=training)

  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  accuracy = tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  
  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
  

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = { "accuracy": accuracy }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=resnet50, model_dir="/model/test1")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=1000)

  pred_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      shuffle=False)
  predict_results = mnist_classifier.predict(input_fn=pred_input_fn)
  print(len(list(predict_results)))
  file = open("prediction.txt","w") 
  for i in predict_results:
    print(str(i['classes']))
    file.write(str(i['classes']))
  file.close()
  
"""
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])
"""
"""
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)
"""
  
if __name__ == "__main__":
  tf.app.run()


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/model/test1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f709a440d68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.


Exception ignored in: <generator object Estimator.predict at 0x7f70c69fbd00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/estimator/estimator.py", line 519, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5265, in get_controller
    yield g
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5068, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /model/test1/model.ckpt-800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
55000


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
